In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import json
import os
from pandas.io.json import json_normalize
import datetime


In [2]:
# Reading the events and guests files from CSV
events_Fathers_Day_df = pd.read_csv('events_Fathers_Day.csv')
guests_host_Fathers_Day_df = pd.read_csv('guests_host_Fathers_Day.csv')

guests_host_Fathers_Day_df = pd.merge(guests_host_Fathers_Day_df,events_Fathers_Day_df[[
    'event_id','premium_vs_free']],on = 'event_id')

In [3]:
def guest2host_ratio(df_events, df_guests):
    temp = df_events[['event_date','host_id']]
    hosts_with_earliest_event_date_df = temp.groupby('host_id', as_index=False).agg(min).rename(columns={'host_id': 'user_id','event_date': 'as_host_date'})
    
    temp = df_guests[['event_date','user_id']]
    all_users_with_earliest_event_date_df = temp.groupby(
        'user_id', as_index=False).agg(min).rename(columns={'event_date': 'all_events_date'})
    
    merged_data = pd.merge(hosts_with_earliest_event_date_df, all_users_with_earliest_event_date_df,\
         on = 'user_id')
    
    merged_data['guest2host'] = (merged_data['as_host_date'] > merged_data['all_events_date']).astype(int)
    merged_data = merged_data.rename(columns={'user_id': 'host_id'})
    
    guest_turned_host = merged_data['guest2host'].astype(int)
    
    return(guest_turned_host.sum()/df_guests['user_id'].nunique())

In [9]:
guest2host_ratio(events_Fathers_Day_df, guests_host_Fathers_Day_df)

0.0011608477706446223

In [4]:
guest2host_ratio(events_Fathers_Day_df[events_Fathers_Day_df['premium_vs_free'] == 'Free'], 
                 guests_host_Fathers_Day_df[guests_host_Fathers_Day_df['premium_vs_free'] == 'Free'])

0.001175152194324136

In [8]:
events_Fathers_Day_df[events_Fathers_Day_df['premium_vs_free'] == 'Free']['premium_vs_free'].shape

(11046,)

In [ ]:
# design a new feature of guests who turned into hosts.
# first: in the events df, for each host find the first date when they hosted an event
# second: in the guests df, for each user find the first date when they attended an event
# third: merge these two tables based on the user_id. you'll get a table with three columns:
# user_id, as_host_date, all_events_date
# in the resulting table, keep only the users for whom the as_host_date is later (more than) all_events_date
# this will give a list of the guests who later became hosts

# "hosts_with_earliest_event_date_df" contains id of each host with the earliest date they hosted an event
temp = events_Fathers_Day_df[['event_date','host_id']]
hosts_with_earliest_event_date_df = temp.groupby('host_id', as_index=False).agg(min).rename(columns={'host_id': 'user_id','event_date': 'as_host_date'})

# "all_users_with_earliest_event_date_df" will contain information of users with an earliest even they attended, including as a guest
temp = guests_host_Fathers_Day_df[['event_date','user_id']]
all_users_with_earliest_event_date_df = temp.groupby('user_id', as_index=False).agg(min).rename(columns={'event_date': 'all_events_date'})

merged_data = pd.merge(hosts_with_earliest_event_date_df, all_users_with_earliest_event_date_df,\
         on = 'user_id')
#merged_data.head()

merged_data['guest2host'] = (merged_data['as_host_date'] > merged_data['all_events_date']).astype(int)
merged_data = merged_data.rename(columns={'user_id': 'host_id'})

guest_turned_host = merged_data['guest2host'].astype(int)

In [10]:
20/10000

0.002